In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, LLAVA_QWEN2, VLM_IMAGE_TOKENS, LlavaQwen2_process_fn 
from src.utils.basic_utils import batch_to_device
from PIL import Image
import torch

/opt/conda/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM2Vec/src/model/baseline_backbone/internvideo2/modeling_internvideo2.py:539: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


DropoutAddRMSNorm of flash_attn is not installed!!!


In [ ]:
model_args = ModelArguments(
    model_name='apple/FastVLM-0.5B',
    pooling='last',
    normalize=True,
    model_backbone='llava_qwen2',
    # lora=True
)
data_args = DataArguments()
processor = load_processor(model_args, data_args)
model = MMEBModel.load(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()

In [9]:
inputs = processor(text=f'{VLM_IMAGE_TOKENS[LLAVA_QWEN2]} Represent the given image with the following question: What is in the image',
                   images=Image.open('assets/example.jpg'),
                   return_tensors="pt")
inputs = {key: value.to('cuda') for key, value in inputs.items()}
print(inputs['input_ids'].shape)
print(inputs['images'].shape)

torch.Size([1, 15])
torch.Size([1, 3, 1024, 1024])


In [11]:
with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    image_features = model.encoder.encode_images(inputs['images'])
print(image_features.shape)

torch.Size([1, 256, 896])


In [ ]:
vision_tower = model.encoder.get_vision_tower()


In [17]:
# xem toàn bộ config trong model
print(vision_tower.__dict__)


{'training': False, '_parameters': {}, '_buffers': {}, '_non_persistent_buffers_set': set(), '_backward_pre_hooks': OrderedDict(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_hooks_with_kwargs': OrderedDict(), '_forward_hooks_always_called': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_forward_pre_hooks_with_kwargs': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_post_hooks': OrderedDict(), '_modules': {'vision_tower': MCi(
  (model): FastViT(
    (patch_embed): Sequential(
      (0): MobileOneBlock(
        (se): Identity()
        (activation): GELU(approximate='none')
        (reparam_conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      )
      (1): MobileOneBlock(
        (se): Identity()
        (activation): GELU(approximate='none')
        (reparam_conv): Conv2d(96, 96, kernel

In [22]:
vision_tower.cfg_only['image_cfg']['patch_size']

64

In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, QWEN2_VL, VLM_IMAGE_TOKENS, Qwen2_VL_process_fn
from src.utils.basic_utils import batch_to_device
from PIL import Image
import torch

model_args = ModelArguments(
    model_name='Qwen/Qwen2-VL-7B-Instruct',
    checkpoint_path='TIGER-Lab/VLM2Vec-Qwen2VL-7B',
    pooling='last',
    normalize=True,
    model_backbone='qwen2_vl',
    lora=True
)
data_args = DataArguments()

processor = load_processor(model_args, data_args)
model = MMEBModel.load(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()


/opt/conda/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM2Vec/src/model/baseline_backbone/internvideo2/modeling_internvideo2.py:539: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


DropoutAddRMSNorm of flash_attn is not installed!!!


[2025-11-29 03:23:26,277] INFO [src.utils.basic_utils:21] Loading processor from: TIGER-Lab/VLM2Vec-Qwen2VL-7B
[2025-11-29 03:23:26,280] DEBUG [urllib3.connectionpool:1051] Starting new HTTPS connection (1): huggingface.co:443
[2025-11-29 03:23:26,652] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /TIGER-Lab/VLM2Vec-Qwen2VL-7B/resolve/main/preprocessor_config.json HTTP/11" 307 0
[2025-11-29 03:23:26,664] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-7B/5aec77d4014be33b00aff8f7856865214682f3d9/preprocessor_config.json HTTP/11" 200 0
[2025-11-29 03:23:26,946] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /TIGER-Lab/VLM2Vec-Qwen2VL-7B/resolve/main/tokenizer_config.json HTTP/11" 307 0
[2025-11-29 03:23:26,957] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-7B/5aec77d4014be33b00aff8f7856865214682f3d9/to

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

[2025-11-29 03:23:32,278] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /Qwen/Qwen2-VL-7B-Instruct/resolve/main/generation_config.json HTTP/11" 307 0
[2025-11-29 03:23:32,289] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /api/resolve-cache/models/Qwen/Qwen2-VL-7B-Instruct/eed13092ef92e448dd6875b2a00151bd3f7db0ac/generation_config.json HTTP/11" 200 0
[2025-11-29 03:23:32,293] INFO [src.utils.basic_utils:21] Loading LoRA from TIGER-Lab/VLM2Vec-Qwen2VL-7B
[2025-11-29 03:23:32,570] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /TIGER-Lab/VLM2Vec-Qwen2VL-7B/resolve/main/adapter_config.json HTTP/11" 307 0
[2025-11-29 03:23:32,580] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /api/resolve-cache/models/TIGER-Lab/VLM2Vec-Qwen2VL-7B/5aec77d4014be33b00aff8f7856865214682f3d9/adapter_config.json HTTP/11" 200 0
[2025-11-29 03:24:06,088] DEBUG [urllib3.connectionpool:546] https://huggingface.co:443 "HEAD /TIGER-

MMEBModel(
  (encoder): PeftModel(
    (base_model): LoraModel(
      (model): Qwen2VLForConditionalGeneration(
        (visual): Qwen2VisionTransformerPretrainedModel(
          (patch_embed): PatchEmbed(
            (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
          )
          (rotary_pos_emb): VisionRotaryEmbedding()
          (blocks): ModuleList(
            (0-31): 32 x Qwen2VLVisionBlock(
              (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
              (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
              (attn): VisionFlashAttention2(
                (qkv): Linear(in_features=1280, out_features=3840, bias=True)
                (proj): Linear(in_features=1280, out_features=1280, bias=True)
              )
              (mlp): VisionMlp(
                (fc1): Linear(in_features=1280, out_features=5120, bias=True)
                (act): QuickGELUActivation()
                (fc2): Linear(i

In [2]:
# Image + Text -> Text
inputs = processor(text=f'{VLM_IMAGE_TOKENS[QWEN2_VL]} Represent the given image with the following question: What is in the image',
                   images=Image.open('assets/example.jpg'),
                   return_tensors="pt")
inputs = {key: value.to('cuda') for key, value in inputs.items()}
inputs.keys()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])